In [1]:
import numpy as np
import TensorFrost as tf
import matplotlib.pyplot as plt

tf.initialize(tf.opengl)

test_axis = -1

def PrefixSum(A, axis = -1):
    axis = len(A.shape) + axis if axis < 0 else axis
    group_size = 128
    grouped = tf.split_dim(A, group_size, axis)
    group_scan = tf.prefix_sum(tf.sum(grouped, axis = axis + 1), axis = axis)
    ids = grouped.indices
    gid, eid = ids[axis], ids[axis + 1]
    ids = [ids[i] for i in range(len(ids)) if i != axis + 1]
    ids[axis] = gid - 1
    group_scan = tf.prefix_sum(grouped + tf.select((gid == 0) | (eid != 0), 0, group_scan[tuple(ids)]), axis = axis + 1)
    full_scan = tf.merge_dim(group_scan, target_size = A.shape[axis], axis = axis + 1)
    return full_scan

def Scan():
    data = tf.input([-1, -1, -1], tf.int32)
    return PrefixSum(data, axis = test_axis)

scan_program = tf.compile(Scan)

TensorFrost module loaded!
Scan:
  Kernel count: 4
  Intermediate buffers: 3
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 516
  IR Compile time: 6.016800 ms
  Codegen time: 1.004700 ms
  Host Compile time: 1455.829956 ms
  Shader Compile time: 0.699500 ms



In [2]:
# Generate some random data to scan (ints between 0 and 10)
data = np.random.randint(0, 10, (25, 61,153), dtype=np.int32)

data_tf = tf.tensor(data)
scan_tf = scan_program(data_tf)

# do scan in numpy
scan_np = np.cumsum(data, axis=test_axis)

#print error
print("Error: ", np.max(np.abs(scan_tf.numpy - scan_np)))

print("Data: ", data.flatten())
print("Diff: ", np.abs(scan_tf.numpy - scan_np).flatten())

ValueError: operands could not be broadcast together with shapes (25,153,128) (25,61,153) 